### Word2Vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/datasets/model.bin', binary=True)

In [ ]:
model.vocab

In [ ]:
mom = model.get_vector('мама_NOUN')

In [ ]:
wash = model.get_vector('мыть_VERB')

In [ ]:
model.cosine_similarities(mom, wash.reshape((-1, 1)))

In [ ]:
queen = model.get_vector('королева_NOUN')
king = model.get_vector('король_NOUN')

man = model.get_vector('мужчина_NOUN')
woman = model.get_vector('женщина_NOUN')

In [ ]:
model.get_vector('кек_NOUN')

In [ ]:
model.similar_by_vector(king - man + woman)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('король_NOUN', 0.8779481649398804),
 ('герцог_NOUN', 0.7090673446655273),
 ('король_PROPN', 0.6953890323638916),
 ('королева_NOUN', 0.67110276222229),
 ('королева_ADV', 0.645533561706543),
 ('карл_PROPN', 0.6311907172203064),
 ('максимилиан_PROPN', 0.6286245584487915),
 ('дофин_NOUN', 0.6282011270523071),
 ('принц_NOUN', 0.6229782104492188),
 ('королевство_NOUN', 0.6215177774429321)]

### CBOW https://github.com/FraLotito/pytorch-continuous-bag-of-words/blob/master/cbow.py

In [ ]:
import torch
import torch.nn as nn

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()



In [ ]:
# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [ ]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [ ]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

#TRAINING
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

#TESTING
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['People', 'create', 'to', 'direct']

Prediction: programs


## Skip-Gram https://github.com/blackredscarf/pytorch-SkipGram

In [ ]:
import torch
from torch import nn


class SkipGramNeg(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(SkipGramNeg, self).__init__()
        self.input_emb = nn.Embedding(vocab_size, emb_dim)
        self.output_emb = nn.Embedding(vocab_size, emb_dim)
        self.log_sigmoid = nn.LogSigmoid()

        initrange = (2.0 / (vocab_size + emb_dim)) ** 0.5  # Xavier init
        self.input_emb.weight.data.uniform_(-initrange, initrange)
        self.output_emb.weight.data.uniform_(-0, 0)


    def forward(self, target_input, context, neg):
        """
        :param target_input: [batch_size]
        :param context: [batch_size]
        :param neg: [batch_size, neg_size]
        :return:
        """
        # u,v: [batch_size, emb_dim]
        v = self.input_emb(target_input)
        u = self.output_emb(context)
        # positive_val: [batch_size]
        positive_val = self.log_sigmoid(torch.sum(u * v, dim=1)).squeeze()

        # u_hat: [batch_size, neg_size, emb_dim]
        u_hat = self.output_emb(neg)
        # [batch_size, neg_size, emb_dim] x [batch_size, emb_dim, 1] = [batch_size, neg_size, 1]
        # neg_vals: [batch_size, neg_size]
        neg_vals = torch.bmm(u_hat, v.unsqueeze(2)).squeeze(2)
        # neg_val: [batch_size]
        neg_val = self.log_sigmoid(-torch.sum(neg_vals, dim=1)).squeeze()

        loss = positive_val + neg_val
        return -loss.mean()

    def predict(self, inputs):
        return self.input_emb(inputs)

## FastText https://towardsdatascience.com/deep-learning-for-nlp-with-pytorch-and-torchtext-4f92d69052f

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from torchtext.data import Field 
from torchtext.data import Dataset, Example
from torchtext.data import BucketIterator
from torchtext.vocab import FastText
from torchtext.vocab import CharNGram

import pandas as pd
import numpy as np

In [ ]:
embedding = FastText('simple')

In [ ]:
# embedding_charngram = CharNGram()

In [ ]:
df = pd.DataFrame([
    ['my name is Jack', 'Y'],
    ['Hi I am Jack', 'Y'],
    ['Hello There!', 'Y'],
    ['Hi I am cooking', 'N'],
    ['Hello are you there?', 'N'],
    ['There is a bird there', 'N'],
], columns=['text', 'label'])
df

,text,label
0,my name is Jack,Y
1,Hi I am Jack,Y
2,Hello There!,Y
3,Hi I am cooking,N
4,Hello are you there?,N
5,There is a bird there,N


In [ ]:
text_field = Field(
    sequential=True,
    tokenize='basic_english', 
    fix_length=5,
    lower=True
)

label_field = Field(sequential=False, use_vocab=False)

# sadly have to apply preprocess manually
preprocessed_text = df['text'].apply(
    lambda x: text_field.preprocess(x)
)

# load fastext simple embedding with 300d
text_field.build_vocab(
    preprocessed_text, 
    vectors='fasttext.simple.300d'
)

# get the vocab instance
vocab = text_field.vocab

In [ ]:
# known token, in my case print 12
print(vocab['are'])
# unknown token, will print 0
print(vocab['crazy'])

12
0


In [ ]:
for i, r in df.iterrows():
    print(list(r.values))

['my name is Jack', 'Y']
['Hi I am Jack', 'Y']
['Hello There!', 'Y']
['Hi I am cooking', 'N']
['Hello are you there?', 'N']
['There is a bird there', 'N']


In [ ]:
# we still have to manually handle conversion from categorical to int
ltoi = {l: i for i, l in enumerate(df['label'].unique())}
df['label'] = df['label'].apply(lambda y: ltoi[y])

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

In [ ]:
train_dataset, test_dataset = DataFrameDataset(
    df=df, 
    fields=(
        ('text', text_field),
        ('label', label_field)
    )
).split()

In [ ]:
train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, test_dataset), 
    batch_sizes=(2, 2),
    sort=False
)

In [ ]:
class ModelParam(object):
    def __init__(self, param_dict: dict = dict()):
        self.input_size = param_dict.get('input_size', 0)
        self.vocab_size = param_dict.get('vocab_size')
        self.embedding_dim = param_dict.get('embedding_dim', 300)
        self.target_dim = param_dict.get('target_dim', 2)
        
class MyModel(nn.Module):
    def __init__(self, model_param: ModelParam):
        super().__init__()
        self.embedding = nn.Embedding(
            model_param.vocab_size, 
            model_param.embedding_dim
        )
        self.lin = nn.Linear(
            model_param.input_size * model_param.embedding_dim, 
            model_param.target_dim
        )
        
    def forward(self, x):
        features = self.embedding(x).view(x.size()[0], -1)
        features = F.relu(features)
        features = self.lin(features)
        return features

In [ ]:
class MyModelWithPretrainedEmbedding(nn.Module):
    def __init__(self, model_param: ModelParam, embedding):
        super().__init__()
        self.embedding = embedding
        self.lin = nn.Linear(
            model_param.input_size * model_param.embedding_dim, 
            model_param.target_dim
        )
        
    def forward(self, x):
        features = self.embedding[x].reshape(x.size()[0], -1)
        features = F.relu(features)
        features = self.lin(features)
        return features

In [ ]:
model_param = ModelParam(
    param_dict=dict(
        vocab_size=len(text_field.vocab),
        input_size=5
    )
)
model = MyModel(model_param)
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)
epochs = 5


for epoch in range(epochs):
    epoch_losses = list()
    for batch in train_iter:
        optimizer.zero_grad()

        prediction = model(batch.text.T)
        loss = loss_function(prediction, batch.label)

        loss.backward()
        optimizer.step()
        
        epoch_losses.append(loss.item())
    print('train loss on epoch {} : {:.3f}'.format(epoch, np.mean(epoch_losses)))
    
    test_losses = list()
    for batch in test_iter:
        with torch.no_grad():
            optimizer.zero_grad()
            prediction = model(batch.text.T)
            loss = loss_function(prediction, batch.label)
            
            test_losses.append(loss.item())
    
    print('test loss on epoch {}: {:.3f}'.format(epoch, np.mean(test_losses)))

train loss on epoch 0 : 0.439
test loss on epoch 0: 5.784
train loss on epoch 1 : 0.002
test loss on epoch 1: 8.804
train loss on epoch 2 : 0.000
test loss on epoch 2: 11.002
train loss on epoch 3 : 0.000
test loss on epoch 3: 12.678
train loss on epoch 4 : 0.000
test loss on epoch 4: 13.999
